In [45]:
from openai import OpenAI
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel
import re, json, os
from typing import Optional, Type, TypeVar, List, Dict, Any
import llm_qa_data_builder
from datetime import datetime
# import pandas as pd

In [46]:
# %pip install openai

Test chat and prompt

In [47]:
!which python

/home/ubuntu/anaconda3/bin/python


In [48]:
def load_azure_config(config_path="azure_config.json"):
    """Load Azure OpenAI configuration from JSON file."""
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        return config
    except FileNotFoundError:
        print(f"Configuration file {config_path} not found. Please create it with your Azure OpenAI settings.")
        return None
    except json.JSONDecodeError:
        print(f"Error parsing {config_path}. Please check the JSON format.")
        return None

# Load configuration
config = load_azure_config()
if not config:
    raise Exception("Failed to load Azure configuration")

In [ ]:
import os
from openai import AzureOpenAI

# model = "gpt-4o-mini-chat"
# endpoint = config['azure_openai']['endpoint']
# model_name = config['models'][model]['model_name']
# deployment = config['models'][model]['deployment_name']

# subscription_key = config['azure_openai']['api_key']
# api_version = config['azure_openai']['api_version']

# print(f"API Key loaded: {subscription_key is not None}")


endpoint = 'endpoint'  # Replace with your Azure OpenAI endpoint
subscription_key = ''
api_version = '2024-05-01-preview'
deployment = 'DeepSeek-R1-0528'

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=0.7,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

<think>
Okay, the user is asking for Paris recommendations - a classic travel question! They didn't specify much about their interests or trip duration, so I should cover the essentials while keeping it flexible. 

Hmm, judging by the simple query, they might be a first-time visitor. Probably wants the iconic experiences but I'll sneak in some local gems too. Should emphasize that Paris is overwhelming - they'll need prioritization help. 

First, the non-negotiables: Eiffel Tower (duh), Louvre (but warn about crowds), Notre-Dame (mention the fire recovery). Arc de Triomphe view is underrated - that Champs-Élysées vista! 

Wait, they might not know about museum passes... must include that money/time-saving tip. And the Seine cruises - romantic but also practical for tired feet. 

For neighborhoods: Le Marais for history hipsters, Montmartre for instagrammers, Saint-Germain for bookish types. Should mention the Latin Quarter's student vibe too. 

Food section is crucial - but don't overw

In [103]:
# # Modify OpenAI's API key and API base to use vLLM's API server.
# openai_api_key = "EMPTY"
# openai_api_base = "http://host.docker.internal:8000/v1"
# client = OpenAI(
#     api_key=openai_api_key,
#     base_url=openai_api_base,
# )
# model = 'Qwen/QwQ-32B-AWQ'

In [104]:
# # Round 1
# messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
# # For granite, add: `extra_body={"chat_template_kwargs": {"thinking": True}}`
# response = client.chat.completions.create(model=model, messages=messages)

# reasoning_content = response.choices[0].message.reasoning_content
# content = response.choices[0].message.content

# print("reasoning_content:", reasoning_content)
# print("content:", content)

In [38]:
response = client.chat.completions.create(
    messages=[{"role": "system", "content": "You are a medical assistant with expertise in understanding clinical timelines and temporal relationships between medical events in clinical text."}, {"role": "user", "content": "\nGiven the clinical text, identify the precise date or time when events marked with <E#> and </E#> occurred.\n\nIMPORTANT FORMATTING INSTRUCTIONS:\n\n    - For specific dates: Use \"ON YYYY-MM-DD\" (e.g., \"ON 2023-05-15\")\n\n    - For approximate dates: Use \"IN YYYY\" for years or \"IN YYYY-MM\" for months (e.g., \"IN 1991\", \"IN 2023-05\")\n\n    - For relative timings: Use \"BEFORE YYYY-MM-DD\" or \"AFTER YYYY-MM-DD\"\n\n    - For ranges: Use \"YYYY-MM-DD TO YYYY-MM-DD\"\n\n    - If there is no temporal information: Use \"None\"\n\n    - Always select the most precise format possible, but do not guess or over-specify if the text does not support it.\n\nKEY PRINCIPLES: \n- For relative references (such as \"yesterday\", \"last week\", \"last winter\"), interpret them using a reference date.\n- For approximate dates (such as \"around May\"), use the most specific form you can justify.\n- If several dates are possible, choose the one most relevant to the clinical context.\n\nTIMESTAMP TAGS: Look for <T0>, <T1>, etc. as temporal references (e.g., <T0>September 15, 2023</T0>).\nReturn your answer in JSON format, like this:\n\n{\n  \"datetime\": \"[formatted date/time]\",\n  \"clues\": \"[specific text evidence supporting your decision]\"\n}\n\nExamples:\n\n    {\"datetime\": \"ON 2023-05-15\", \"clues\": \"procedure performed on May 15, 2023\"}\n\n    {\"datetime\": \"IN 2022\", \"clues\": \"The patient was admitted in 2024, with a diabetes diagnosis two years ago\"}\n\n    {\"datetime\": \"IN 1992-10\", \"clues\": \"CT scan performed in October 1992\"}\n\n    {\"datetime\": \"BEFORE 2023-05-15\", \"clues\": \"symptoms started prior to May 15, 2023\"}\n\n    {\"datetime\": \"2023-05-15 TO 2023-05-20\", \"clues\": \"treatment from May 15 through May 20, 2023\"}\n\n    {\"datetime\": \"None\", \"clues\": \"no temporal information found\"}\n    \n\n\nCLINICAL TEXT: Admission Date :\n09/29/1993\nDischarge Date :\n10/04/1993\nHISTORY OF PRESENT ILLNESS :\nThe patient is a 28-year-old woman who is <E2>HIV positive</E2> for two years .\nShe presented with <E4>left upper quadrant pain</E4> as well as <E5>nausea</E5> and <E6>vomiting</E6> which is <E7>a long-standing complaint</E7> .\nShe was diagnosed in 1991 during the birth of her child .\nShe claims she does not know why she is <E10>HIV positive</E10> .\nShe is from Maryland , apparently had no <E11>blood transfusions</E11> before the birth of her children so it is presumed <E13>heterosexual transmission</E13> .\nAt that time , she also had <E14>cat scratch fever</E14> and she had <E15>resection</E15> of <E16>an abscess in the left lower extremity</E16> .\nShe has not used <E17>any anti retroviral therapy</E17> since then , because of <E18>pancytopenia</E18> and <E19>vomiting</E19> on <E20>DDI</E20> .\nShe has complaints of <E21>nausea</E21> and <E22>vomiting</E22> as well as <E23>left upper quadrant pain</E23> on and off getting progressively worse over the past month .\nShe has had <E25>similar pain</E25> intermittently for last year .\nShe described <E26>the pain</E26> as <E27>a burning pain</E27> which is positional , worse when she walks or does any type of exercise .\nShe has no relief from <E32>antacids</E32> or <E33>H2 blockers</E33> .\nIn 10/92 , she had <E34>a CT scan</E34> which showed <E35>fatty infiltration of her liver diffusely</E35> with <E36>a 1 cm cyst in the right lobe of the liver</E36> .\nShe had a normal pancreas at that time , however , <E38>hyperdense kidneys</E38> .\n<E39>Her alkaline phosphatase</E39> was <E40>slightly elevated</E40> but otherwise relatively normal .\n<E42>Her amylase</E42> was <E43>mildly elevated</E43> but has been down since then .\nThe patient has had <E45>progressive failure to thrive</E45> and <E46>steady weight loss</E46> .\nShe was brought in for <E47>an esophagogastroduodenoscopy</E47> on 9/26 but she basically was not sufficiently <E48>sedated</E48> and readmitted at this time for <E50>a GI work-up</E50> as well as <E51>an evaluation</E51> of <E52>new abscess in her left lower calf</E52> and right medial lower extremity quadriceps muscle .\nShe was also admitted to be connected up with social services for <E55>HIV</E55> patients .\nHOSPITAL COURSE :\nThe patient was admitted and <E57>many cultures</E57> were sent which were all negative .\nShe did not have any of <E59>her pain</E59> in the hospital .\nOn the third hospital day , she did have <E61>some pain</E61> and was treated with <E62>Percocet</E62> .\nShe went for <E63>a debridement</E63> of <E64>her left calf lesion</E64> on 10/2/93 and was started empirically on <E65>IV ceftriaxone</E65> which was changed to <E66>po doxycycline</E66> on the day of discharge .\n<E68>A follow-up CT scan</E68> was done which did not show any <E70>evidence</E70> for <E71>splenomegaly</E71> or <E72>hepatomegaly</E72> .\n<E73>The 1 cm cyst</E73> which was seen in 10/92 was still present .\nThere was a question of <E74>a cyst in her kidney</E74> with <E75>a stone</E75> right below <E76>the cyst</E76> , although this did not seem to be clinically significant ."}],
    temperature=0.2,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

```json
{
  "events": [
    {
      "datetime": "ON 1991-00-00",
      "clues": "She was diagnosed in 1991 during the birth of her child."
    },
    {
      "datetime": "ON 1992-10-00",
      "clues": "In 10/92, she had a CT scan which showed fatty infiltration of her liver."
    },
    {
      "datetime": "ON 1993-09-26",
      "clues": "She was brought in for an esophagogastroduodenoscopy on 9/26."
    },
    {
      "datetime": "ON 1993-10-02",
      "clues": "She went for a debridement of her left calf lesion on 10/2/93."
    },
    {
      "datetime": "ON 1993-09-29",
      "clues": "Admission Date: 09/29/1993."
    },
    {
      "datetime": "ON 1993-10-04",
      "clues": "Discharge Date: 10/04/1993."
    },
    {
      "datetime": "None",
      "clues": "No specific date for the patient's long-standing complaints of nausea and vomiting."
    },
    {
      "datetime": "None",
      "clues": "No specific date for the patient's progressive failure to thrive and steady weight los

## Error analysis

In [41]:
def get_gold_standard(folder='timeline_training'):
    # path = '/home/ubuntu/work/Temporal_relation/'
    path = '/home/ubuntu/zhaoyue/Temporal_relation/'
    data_dir = path + "data/" + folder + "/"
    print(f"Loading data from {data_dir}")
    results = {}

    data = llm_qa_data_builder.data_load(data_dir, True)

    for key in data:
        ground_truth = {}
        text = data[key]['label']
        event_start_time = data[key]['starttime']
        for event in event_start_time:
            ground_truth[event['node_id']] = {
                "event_id": event['node_id'],
                "formatted_time_range": event['formatted_time_range'],
            }
        # print(ground_truth)
        results[key] = ground_truth
    return results

In [42]:
folder1 = 'timeline_training_llm_update'
folder2 = 'timeline_training'

gs1 = get_gold_standard(folder1)
gs2 = get_gold_standard(folder2)

Loading data from /home/ubuntu/zhaoyue/Temporal_relation/data/timeline_training_llm_update/
Loaded data for 190 IDs.
Loading data from /home/ubuntu/zhaoyue/Temporal_relation/data/timeline_training/
Loaded data for 190 IDs.


In [53]:
# gs1

In [54]:
event_count = [len(gs1[key]) for key in gs1]
print(f"average number of events in {folder1}: {sum(event_count) / len(event_count)}")

average number of events in timeline_training_llm_update: 58.084210526315786


In [56]:
# event_count

In [44]:
for docid in gs1.keys():
    if docid not in gs2:
        print(f"Document {docid} is missing in {folder2}")
    else:
        for event_id, event_data in gs1[docid].items():
            # print(event_data['formatted_time_range'], gs2[docid][event_id]['formatted_time_range'])
            # print(event_id, event_data)
            if event_id not in gs2[docid]:
                print(f"Event {event_id} is missing in {folder2} for document {docid}")
            elif event_data['formatted_time_range'] != gs2[docid][event_id]['formatted_time_range']:
                print(f"Event {event_id} has different ground truth in {folder1} and {folder2} for document {docid}")
            # else:
            #     print(f"Event {event_id} matches in both folders for document {docid}")

Event E72 has different ground truth in timeline_training_llm_update and timeline_training for document 1
Event E22 has different ground truth in timeline_training_llm_update and timeline_training for document 107
Event E76 has different ground truth in timeline_training_llm_update and timeline_training for document 107
Event E31 has different ground truth in timeline_training_llm_update and timeline_training for document 107
Event E6 has different ground truth in timeline_training_llm_update and timeline_training for document 11
Event E55 has different ground truth in timeline_training_llm_update and timeline_training for document 11
Event E61 has different ground truth in timeline_training_llm_update and timeline_training for document 11
Event E60 has different ground truth in timeline_training_llm_update and timeline_training for document 11
Event E41 has different ground truth in timeline_training_llm_update and timeline_training for document 11
Event E50 has different ground truth

In [ ]:
# The same data in both folders.

## Build prompts

In [1]:
import json

In [3]:
# common = """
# Given the clinical text, identify the precise date or time when events marked with <E#> and </E#> occurred.

# IMPORTANT FORMATTING INSTRUCTIONS:

#     - For specific dates: Use "ON YYYY-MM-DD" (e.g., "ON 2023-05-15")

#     - For approximate dates: Use "IN YYYY" for years or "IN YYYY-MM" for months (e.g., "IN 1991", "IN 2023-05")

#     - For relative timings: Use "BEFORE YYYY-MM-DD" or "AFTER YYYY-MM-DD"

#     - For ranges: Use "YYYY-MM-DD TO YYYY-MM-DD"

#     - If there is no temporal information: Use "None"

#     - Always select the most precise format possible, but do not guess or over-specify if the text does not support it.

# KEY PRINCIPLES: 
# - For relative references (such as "yesterday", "last week", "last winter"), interpret them using a reference date.
# - For approximate dates (such as "around May"), use the most specific form you can justify.
# - If several dates are possible, choose the one most relevant to the clinical context.

# """



In [18]:
common = """

Given the clinical text, determine the exact start date or time for events enclosed by <E#> and </E#> tags.

The identified start time must strictly follow one of the six allowed formats — no deviations are permitted:

* AT <timestamp> – For specific moments in time (e.g., AT 2002-02-01 14:00:00)
* ON <date> – For precise calendar dates (e.g., ON 2002-02-01)
* IN <month> or IN <year> – For approximate periods (e.g., IN 2002-02, IN 2002)
* BEFORE <timestamp> – If the event occurred before a known time
* AFTER <timestamp> – If the event occurred after a known time
* <timestamp1> TO <timestamp2> – For events spanning a known interval

All timestamps must be in one of the following formats: YYYY-MM-DD HH:MM:SS, YYYY-MM-DD, YYYY-MM, or YYYY.

---

Special Instructions and Edge Cases:

1. **Vague or Approximate Expressions**  
   If the time is inferred from imprecise language (e.g., “several years ago”), provide a best estimate and prefix it with `APPROXIMATELY`. Examples:
   * APPROXIMATELY IN 2001  
   * APPROXIMATELY 1999-01-01 TO 2000-01-01

2. **Negative Events**  
   For events described as *not* occurring, estimate when the absence began — not when it was recorded. Use contextual clues.  
   For example, if a note says, “the patient has been coughing for two years but never had a fever,” the negative event “fever” should be dated to when the cough began — two years ago.

3. **Planned or Future Events**  
   For events scheduled but not yet occurred (e.g., planned procedures), estimate the intended start time. This is typically based on the discharge date or any future timeframe explicitly stated in the document.

"""


In [19]:
individual_output = """
Return your answer in JSON format, like this:

{
  "datetime": "[formatted date/time]",
  "clues": "[specific text evidence supporting your decision]"
}
    
"""

all_output = """Return your answer in JSON format, like this:

[
  {
    "event_id": "[the ID of the event (e.g., "E3", "E5")]",
    "datetime": "[formatted date/time]",
    "clues": "[specific text evidence supporting your decision]"
  }
]
"""

In [20]:
time = "TIMESTAMP TAGS: Look for <T0>, <T1>, etc. as temporal references (e.g., <T0>September 15, 2023</T0>)."
section = """SECTION CONTEXT:
If the text lacks clear temporal markers, use the surrounding section context to help determine the timing of events:
- Events in 'History of Present Illness' or similar sections: use admission dates as reference points.
- Events in 'Hospital Course' or discharge sections: use discharge dates as reference points.
"""

In [21]:
keys = [
  "individual_notime",
  "individual_time",
  "individual_notime_section",  
"individual_time_section",
  "all_notime",
  "all_time",
  "all_notime_section",
  "all_time_section"
]
promots = {}


In [22]:
for k in keys:
    p = common
    if "time" in k:
        p = p + time
    if "section" in k:
        p = p + section
    if "individual" in k:
        p = p + individual_output
    else:
        p = p + all_output
    promots[k] = p

In [23]:
json.dump(promots, open("prompts.json", "w"), indent=2, ensure_ascii=False)

In [25]:
promots

{'individual_notime': '\n\nGiven the clinical text, determine the exact start date or time for events enclosed by <E#> and </E#> tags.\n\nThe identified start time must strictly follow one of the six allowed formats — no deviations are permitted:\n\n* AT <timestamp> – For specific moments in time (e.g., AT 2002-02-01 14:00:00)\n* ON <date> – For precise calendar dates (e.g., ON 2002-02-01)\n* IN <month> or IN <year> – For approximate periods (e.g., IN 2002-02, IN 2002)\n* BEFORE <timestamp> – If the event occurred before a known time\n* AFTER <timestamp> – If the event occurred after a known time\n* <timestamp1> TO <timestamp2> – For events spanning a known interval\n\nAll timestamps must be in one of the following formats: YYYY-MM-DD HH:MM:SS, YYYY-MM-DD, YYYY-MM, or YYYY.\n\n---\n\nSpecial Instructions and Edge Cases:\n\n1. **Vague or Approximate Expressions**  \n   If the time is inferred from imprecise language (e.g., “several years ago”), provide a best estimate and prefix it with